Rédiger un Notebook contenant le MCD, une description du travail de création des bases et le code SQL des vues et des procédures

# MCD

# Description du travail de création des bases

Dans un premier temps nous avons réfléchi à comment découper le dataset pour choisir les différentes tables à créer.

Ensuite nous avons séparé le travail en plusieurs tâches:

- la création de la base de données et des tables
- le traitement des données pour nettoyer des valeurs manquantes, création de nouveaux fichiers csv pour simplifier l'intégration des données et le remplacement des valeurs qualitatives par leurs id respectifs

## Création de la bdd

Nous nous sommes fixés sur une table "principale" et trois tables "indépendantes":
- table *genre* (indépendante): indique l'id et le genre de jeu
- table *publisher* (indépendante): indique l'id et l'éditeur du jeu
- table *plateforme* (indépendante): indique l'id et la plateforme
- table *vente* (principale): va renseigner toutes les entrées restantes
    - id_vente
    - rank : le classement du jeu en fonction de ses ventes dans le monde
    - nom_jeu : nom du jeu
    - id_plateforme
    - annee : année de sortie du jeu
    - id_genre
    - id_publisher
    - NA_sales : ventes en Amérique du nord
    - EU_sales : ventes en Europe
    - JP_sales : ventes au Japon
    - Other_sales : ventes dans les autres pays
    - Global_sales : ventes totales dans le monde

## Traitement des données

#### Importation des bibliothèques 

In [2]:
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
from data_base import Bdd

#### Importation des données

In [3]:
df = pd.read_csv("vgsales_debase.csv")

#### Affichage des information de la bdd

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


On peut voir que l'on a dans la colonne *Year* 271 valeurs sont nulles et que la colonne *Publisher* en a 58.

Pour la colonne *Plublisher*, ça ne pose pas de problèmes, il s'agit de chaînes de caractères du coup on pourra les transformer sans imputer la bdd.

Pour la colonne *Year* ça sera plus compliqué. Les valeurs étant des integer, les remplacer par des string poserait problème tout comme mettre des valeurs standards. On a donc fait le choix de supprimer les lignes correspondantes (cela repésentait une perte de 1,6% des données)

#### Remplacement des valeurs nulles dans Publisher

In [ ]:
df['Publisher'] = df['Publisher'].fillna('unknow')

#### suppression des lignes contenant des Nan (uniquement contenus dans la colonne Year qui seraient compliqués à remplacer)

In [ ]:
df = df.dropna()

#### Remplacement des valeurs par leurs id

### Création des procédures

##### procédure qui affiche les jeux d'un genre donné

````SQL    
    delimiter //
        create procedure jeu_genre 
        (in genres varchar (50))
        begin 
            SELECT nom_jeu, nom_plateforme, annee, nom_genre, nom_publisher, global_sales 
            FROM ventes 
            JOIN genre on genre.id_genre = ventes.id_genre
            JOIN plateforme on plateforme.id_plateforme = ventes.id_plateforme
            JOIN publisher on publisher.id_publisher = ventes.id_publisher
            WHERE nom_genre = genres
            ORDER by global_sales;
        end //
    delimiter ;
    
    call jeu_genre ()
````

##### procédure qui affiche les jeux en fonction de la plateforme

````SQL
    delimiter //
        create procedure jeu_plateforme 
        (in plateformes varchar (50))
        begin 
            SELECT * 
            FROM ventes 
            JOIN plateforme on plateforme.id_plateforme = ventes.id_plateforme
            WHERE plateforme = plateformes;
        end //
    delimiter ; 
    
    call jeu_plateforme ()
````

##### procédure qui affiche les jeux en fonction du publisher

````SQL
    delimiter //
        create procedure jeu_publisher 
        (in publishers varchar (100))
        begin 
            select * 
            from ventes 
            join publisher on publisher.id_publisher = ventes.id_publisher
            where publisher = publishers;
        end //
    delimiter ; 

    call jeu_publisher ()
````

##### affichage des jeux pour une date précise

````SQL    
    delimiter //
        create procedure jeu_date_fixe 
        (in annee_fixe int (4))
        begin 
            select * 
            from ventes 
            where annee = annee_fixe;
        end //
    delimiter ; 
    
    call jeu_date_fixe ()
````

##### affichage des jeux qui sont sortis après une date donnée

````SQL
    delimiter //
        create procedure jeu_date_sup 
        (in annee_sup int (4))
        begin 
            select * 
            from ventes 
            where annee > annee_sup;
        end //
    delimiter ; 
    
    call jeu_date_sup ()
````

##### affichage des jeux qui sont sortis avant une date donnée

````SQL
    delimiter //
        create procedure jeu_date_inf 
        (in annee_inf int (4))
        begin 
            select * 
            from ventes 
            where annee < annee_inf;
        end //
    delimiter ; 
    
    call jeu_date_inf ()
````

##### affichage des jeux dont les ventes globales sont suppérieures à une quantitée donnée

````SQL
    delimiter //
        create procedure jeu_global_sup
        (in global_sup int (4))
        begin 
            select * 
            from ventes 
            where Global_sales > global_sup;
        end //
    delimiter ;
    
    call jeu_global_sup ()
````

##### affichage des jeux dont les ventes globales sont inférieures à une quantitée donnée

````SQL
    delimiter //
        create procedure jeu_global_inf
        (in global_inf int (4))
        begin 
            select * 
            from ventes 
            where Global_sales < global_inf;
        end //
    delimiter ;
    
    call jeu_global_inf ()
````